In [ ]:
from common import *
from snobedo.snotel import CsvParser, SnotelLocations

In [ ]:
client = start_cluster(6, 7)
client

In [ ]:
year = 2020
water_year = f'wy{year}'

melt_start = pd.to_datetime(f'{year}-04-01')

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites_2x2.json')

## SNOTEL

In [ ]:
schofield_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Schofield/usda-csv/{year}-Schofield-Pass.csv',
)

butte_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Butte/usda-csv/{year}-Butte.csv',
)

taylor_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Upper-Taylor/usda-csv/{year}-Upper-Taylor.csv',
)

## iSnobal 

In [ ]:
wy_snow = xr.open_mfdataset(
    (SNOBAL_DIR / water_year / 'runs/*/snow.nc').as_posix(),
    parallel=True,
).sel(time=datetime.time(23))

butte_snobal = wy_snow.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat).coarsen(**COARSEN_OPTS)
schofield_snobal = wy_snow.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat).coarsen(**COARSEN_OPTS)
taylor_snobal = wy_snow.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat).coarsen(**COARSEN_OPTS)

In [ ]:
plot_data = {
    'Butte': {
        'min': butte_snobal.min(),
        'max': butte_snobal.max(),
        'snotel': butte_snotel_csv,
    },
    'Schofield Pass': {
        'min': schofield_snobal.min(),
        'max': schofield_snobal.max(),
        'snotel': schofield_snotel_csv,
    },
    'Upper Taylor': {
        'min': taylor_snobal.min(),
        'max': taylor_snobal.max(),
        'snotel': taylor_snotel_csv,
    },
}

## Compare SNOTEL to iSnobal

In [ ]:
figure_opts = dict(figsize=(8,4), dpi=200, tight_layout=True)
plot_range = pd.date_range(start=f'{year - 1}-10-01', periods=11, freq='MS')
text_props = dict(facecolor='whitesmoke', alpha=0.5, pad=.5, boxstyle='round')

xTicks = mdates.DateFormatter('%d-%b')

def plot_variable(snobal, snotel, title, label, save_figure=False):
    for site in plot_data.keys():
        figure, ax = plt.subplots(1, 1, **figure_opts)
        figure.set_facecolor('lightgrey')
        
        plot_data[site]['min'][snobal].plot(ax=ax, c='slategrey', alpha=0.8)
        plot_data[site]['max'][snobal].plot(ax=ax, c='slategrey', alpha=0.8)
        ax.fill_between(
            plot_data[site]['min'].time, 
            plot_data[site]['min'][snobal].data.flatten(), 
            plot_data[site]['max'][snobal].data.flatten(),
            label='Snobal',
            color='navajowhite', alpha=0.8
        )
        plot_data[site]['snotel'][snotel].plot(
            ax=ax, c='steelblue', label='Snotel'
        )
        
        # Returns LIFO order
        handles, legend = ax.get_legend_handles_labels()
                
        ax.text(
            0.03, 0.95, 
            site,
            style='italic',
            transform=ax.transAxes, 
            verticalalignment='top', 
            bbox=text_props
        )
        
        if title == 'Snow Depth':
            entries = depth_numbers(site, handles[1], handles[0])
            add_legend_box(ax, entries)
        elif title == 'SWE':
            entries = swe_numbers(site, handles[1], handles[0])
            add_legend_box(ax, entries)
        else:
            ax.legend(loc='upper right');
        
        ax.xaxis.set_major_formatter(xTicks)
        ax.set_xlim([plot_range[0], plot_range[-1]])
        ax.set_xlabel(f'Water Year {plot_range[-1].year}')
        ax.set_ylim(bottom=0)
        ax.set_ylabel(label)
        ax.set_title(None);
        
        if save_figure:
            filename = f'{year}_{title}_{site}'.replace(' ', '-')
            plt.savefig(f"{FIGURES_DIR}/{filename}.png")
                
            
def swe_numbers(site, snobal, snotel):
    snobal_min = int(plot_data[site]['min']['specific_mass'].sum())
    snobal_max = int(plot_data[site]['max']['specific_mass'].sum())
    snotel_sum = int(plot_data[site]['snotel']['SWE(mm)'].sum())
    
    legend = []
    legend.append(snobal)
    legend.append(legend_text(' max', f'{snobal_max} mm'))
    legend.append(legend_text(' % Snotel', f'{snobal_max/snotel_sum:.2%}'))
    legend.append(legend_text(' min', f'{snobal_min} mm'))
    legend.append(legend_text(' % Snotel', f'{snobal_min/snotel_sum:.2%}'))
    legend.append(snotel)
    legend.append(legend_text(' total', f'{snotel_sum} mm'))
    
    return legend


def depth_numbers(site, snobal, snotel):
    snobal_depths = plot_data[site]['min']['thickness'].squeeze('x').squeeze('y').to_pandas()
    min_date = snobal_depths[(snobal_depths == 0) & (snobal_depths.index > melt_start)].index[0]
    
    snobal_depths = plot_data[site]['max']['thickness'].squeeze('x').squeeze('y').to_pandas()
    max_date = snobal_depths[(snobal_depths == 0) & (snobal_depths.index > melt_start)].index[0]

    snotel_depth = plot_data[site]['snotel']['Depth(m)']
    snotel_date = snotel_depth[(snotel_depth == 0) & (snotel_depth.index > melt_start)].index[0]

    legend = []
    legend.append(snobal)
    legend.append(legend_text(' Min', min_date.strftime('%Y-%m-%d')))
    legend.append(legend_text(' Max', max_date.strftime(LEGEND_DATE)))

    legend.append(snotel)
    legend.append(legend_text('', snotel_date.strftime(LEGEND_DATE)))
    legend.append(legend_text('Difference:', ''))
    legend.append(legend_text(' Min', f' {(min_date - snotel_date).days} days'))
    legend.append(legend_text(' Max', f' {(max_date - snotel_date).days} days'))
    
    return legend


### Depth 

In [ ]:
plot_variable('thickness', 'Depth(m)', 'Snow Depth', 'Snow Depth (m)', True)

### SWE

In [ ]:
plot_variable('specific_mass', 'SWE(mm)', 'SWE', 'SWE (mm)', True)

### Stats 

## Snow Density

In [ ]:
# plot_variable('snow_density', 'Density(kg/m3)', 'Snow Density', r'Snow Density ($kg/m^3$)')

## Temperature 

In [ ]:
# plot_variable('temp_surf', 'Air-T(C)', 'Temperature', 'Temperatreu (C)')